In [1]:
from data.db_model import *
from data.feature_eng.repetitiveness import * 

import pandas as pd
import numpy as np
import pronouncing
import string, re

engine = create_engine('sqlite:///data/hiphop_rec.db')
session = Session(bind=engine)


# fuzz is used to compare TWO strings
from fuzzywuzzy import fuzz

# process is used to compare a string to MULTIPLE other strings
from fuzzywuzzy import process

/Users/vpatel2/anaconda3/envs/hip-hop/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
df_artists = pd.read_sql('artists', engine)
df_albums = pd.read_sql('albums', engine)
df_songs = pd.read_sql('songs', engine)
df_spotify = pd.read_sql('spotify_song_data', engine)
df_songs_merged = pd.merge(df_songs, df_albums, left_on = 'album_id', right_on = 'id')

,id_x,name_x,album_id,all_lyrics,artist_lyrics,repetitiveness,topic,rhyme,id_y,name_y,artist_id,release_date
23000,30070,Real Nigga No Flex by Waka Flocka Flame & Youn...,2639,"Real nigga, no flex\n Flooded out my rolex\n I...","Real nigga, no flex\n Flooded out my rolex\n I...",0.700991189427313,4,3.934297,2639,Lebron Flocka James 4,182,2016-02-02
23001,30074,FlockaSizzle Freestyle by Waka Flocka Flame & ...,2639,Part that pussy baby tell no told\n Bitch you ...,Don't make me go gorilla with this baby K\n I ...,0.466269841269841,4,1.723517,2639,Lebron Flocka James 4,182,2016-02-02
23002,30080,Timeline,2639,"I’m a fool with that work ,work\n Pull up in t...","I’m a fool with that work ,work\n Pull up in t...",0.717546362339515,0,3.722052,2639,Lebron Flocka James 4,182,2016-02-02
23003,30081,Hype,2639,Pullin' up with a couple shooters\n I could gi...,Pullin' up with a couple shooters\n I could gi...,0.512526096033403,4,6.929571,2639,Lebron Flocka James 4,182,2016-02-02
23004,30082,Chill Baby,2639,"I told that bitch to chill baby, chill baby ei...","I told that bitch to chill baby, chill baby ei...",0.768098159509202,3,4.844314,2639,Lebron Flocka James 4,182,2016-02-02
23005,30084,What's A Party,2639,I just dropped this hand in my sweet sleep (wo...,I just dropped this hand in my sweet sleep (wo...,0.678987341772152,4,7.646043,2639,Lebron Flocka James 4,182,2016-02-02
23006,30085,Jump Around by KSI,2640,Jump around! Jump around! Jump around!\n Jump ...,"Jump up and down, shorty moshpit\n Jump up and...",0.727472527472528,3,1.514953,2640,Waka Flocka Myers 8,182,2016-02-02
23007,30086,Turn Up On Dat by Young Buck,2640,I bought a bunch of pounds and a lot of coke\n...,"I'm in Tennessee, boolin' out in Bashville\n I...",0.702274573517465,4,7.557225,2640,Waka Flocka Myers 8,182,2016-02-02
23008,30087,One Eyed Shooters by Waka Flocka Flame & Young...,2640,She gon' sell her soul just to live another li...,Keep a thirty clip\n Motherfuck yo' petty plot...,0.656897100987576,4,9.173566,2640,Waka Flocka Myers 8,182,2016-02-02
23009,30088,Ball Hard by Waka Flocka Flame & Young Sizzle,2640,"SQUAD Rari's back to back, me & Sizzle\n Throw...","SQUAD Rari's back to back, me & Sizzle\n Throw...",0.626901140684411,4,12.611456,2640,Waka Flocka Myers 8,182,2016-02-02


In [8]:
test = df_songs[df_songs.name == "Ether"].all_lyrics

In [9]:
def split_on_new_line(lyrics):
    return lyrics.split('\n')

In [10]:
def remove_duplicate_lines(lyric):
    all_searches=[]
    for idx, line in enumerate(lyric):
        duplicate_prob = process.extract(line, lyric[idx:], scorer=fuzz.ratio)
        search = [dup[0] for dup in duplicate_prob if dup[1] > 80]
        if len(search) > 1:
            for s in search: 
                all_searches.append(s)
    repeats = list(set(all_searches))
    
    repeat_locations_list=[]
    for repeat in repeats: 
        repeat_is_match = np.isin(lyric, repeat)
        repeat_locations = np.where(repeat_is_match)
        for elem in repeat_locations[0]:
            repeat_locations_list.append(elem)
    for idx in sorted(repeat_locations_list, reverse=True): 
        lyric.pop(idx)
    no_duplicates = lyric
    return no_duplicates

In [11]:
def clean_tokenize(no_duplicates):
    exclude = set(string.punctuation)
    exclude.remove("'")
    regex = re.compile('[%s]' % re.escape(''.join(exclude)))
    
    cleaned= []
    for i in ''.join(lyric).lower().split(' '):
        cleaned.append(regex.sub('', i))
    return cleaned

In [12]:
def score_lyrics(tokenized_lyrics):

    phones_word = [pronouncing.phones_for_word(word) for word in cleaned]
    phones = [phone[0].split(' ') for phone in phones_word if len(phone)>0]
    phones_all = np.array([phone for phone_group in phones for phone in phone_group])
    phones_vowels = np.array([phone[:-1] for phone_group in phones for phone in phone_group])
    
    # phones[np.where(phones == 'EY1')]
    vowels = ['AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'EH', 'ER', 'EY', 'IH', 'IY', 'OW', 'OY', 'UH', 'UW']

    chains_dict = dict((vowel,[]) for vowel in vowels)

    for vowel in vowels:
        is_vowel_match = np.isin(phones_vowels, vowel)
        chain = np.where(is_vowel_match)[0]
        diff = np.array([(chain[idx+1]-indice) for idx,indice in enumerate(chain) if idx+1 < len(chain)])
        diff_ind = np.where( diff>15)
        chains_all = np.split(chain, diff_ind[0] + 1)
        chains = np.array([chain for chain in chains_all if len(chain) > 1])
        chains_dict[vowel] = chains

    score = 0
    for key in chains_dict.keys():
        for chain in chains_dict[key]:
            score += (np.exp(len(chain))/len(phones_all))
    return np.log(score)

In [13]:
df_songs.rhyme.isna().sum()

23369

In [17]:
for idx, row in df_songs.iterrows():
    lyric = split_on_new_line(row['all_lyrics'])
    no_duplicates = remove_duplicate_lines(lyric)
    cleaned = clean_tokenize(no_duplicates)
    score = score_lyrics(cleaned)
    song = session.query(Song).filter(Song.id == row['id'])[0]
    print(song.name, song.rhyme)
    song.rhyme = score
    session.commit()

​a lot 10.025322123964983


​break da law 3.7839848574459976


​a&t 4.120655341356259


​gun smoke 8.13357050475933


1.5 4.552972650805814


...

Tycoon None


Broke Bitches None


Flash On These Bitches None


Too Many None


KeyboardInterrupt: 

In [277]:
session.commit()

In [25]:
lyric = split_on_new_line(df_songs[df_songs.id==7094].all_lyrics.values[0])
no_duplicates = remove_duplicate_lines(lyric)
cleaned = clean_tokenize(no_duplicates)
score = score_lyrics(cleaned)

In [26]:
score

48.599149525634914

In [33]:
df_songs.isna().sum()

id                    0
name                  0
album_id              0
all_lyrics            0
artist_lyrics         0
repetitiveness        0
topic                 0
rhyme             17617
dtype: int64

In [1]:
from data.fetch.scrape_genius import *

In [1]:
from data.db_model import *
from data.feature_eng.repetitiveness import * 

import pandas as pd
import numpy as np
import pronouncing
import string, re

engine = create_engine('sqlite:///data/hiphop_rec.db')
session = Session(bind=engine)


# fuzz is used to compare TWO strings
from fuzzywuzzy import fuzz

# process is used to compare a string to MULTIPLE other strings
from fuzzywuzzy import process

/anaconda3/envs/genres/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [7]:
from data.clean_data import *
from data.master import *

In [8]:
engine = create_engine('sqlite:///data/hiphop_rec.db')
session = Session(bind=engine)

In [9]:
dups = get_duplicates(session, df_songs_merged)

In [10]:
dups

[[('This Life Forever', 'Jay-Z', 11937),
  ('This Life Forever', '21 Savage', 30580)]]

In [11]:
len(df_songs_merged)

23897

In [12]:
df_songs_merged[23850:]

,id_x,name_x,album_id,all_lyrics,artist_lyrics,repetitiveness,topic,rhyme,id_y,name_y,artist_id,release_date
23850,31010,Family Feud,2714,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,None,NaN,NaN,2714,4:44,67,2017-06-30
23851,31011,Bam,2714,Now greetings to the world\n Gangsta nuh live ...,Now greetings to the world\n Gangsta nuh live ...,None,NaN,NaN,2714,4:44,67,2017-06-30
23852,31012,Moonlight,2714,"Yeah, got a lil' cold\n So bear with me\n Yeah...","Look, I know killers, you no killer, huh?\n Ba...",None,NaN,NaN,2714,4:44,67,2017-06-30
23853,31013,Marcy Me,2714,Live from Bedford-Stuyvesant\n The livest one ...,Live from Bedford-Stuyvesant\n The livest one ...,None,NaN,NaN,2714,4:44,67,2017-06-30
23854,31014,Legacy,2714,"Daddy, what's a will?\n Take those monies and ...",Take those monies and spread 'cross families\n...,None,NaN,NaN,2714,4:44,67,2017-06-30
23855,31015,Adnis,2714,"Uh, letter to my dad that I never wrote\n Spee...","Uh, letter to my dad that I never wrote\n Spee...",None,NaN,NaN,2714,4:44,67,2017-06-30
23856,31016,Blue's Freestyle/We Family,2714,"Everything, everything this my only single thi...","I got bloodlines in Benin, that explains the v...",None,NaN,NaN,2714,4:44,67,2017-06-30
23857,31017,MaNyfaCedGod,2714,"Yeah, uh\n Vacay the pain away, uh\n Smoking t...","Yeah, uh\n Vacay the pain away, uh\n Smoking t...",None,NaN,NaN,2714,4:44,67,2017-06-30
23858,31018,4:44 [Credits],2714,"1. “Kill Jay Z”\n (S. Carter, D. Wilson, A. Pa...","1. “Kill Jay Z”\n (S. Carter, D. Wilson, A. Pa...",None,NaN,NaN,2714,4:44,67,2017-06-30
23859,31019,Holy Grail,2715,You'd take the clothes off my back and I'd let...,You'd take the clothes off my back and I'd let...,None,NaN,NaN,2715,Magna Carta... Holy Grail,67,2013-07-04


In [15]:
df_songs_merged.query("name_x == 'Family Feud'")

,id_x,name_x,album_id,all_lyrics,artist_lyrics,repetitiveness,topic,rhyme,id_y,name_y,artist_id,release_date
9104,11890,Family Feud,997,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,0.579360100376412,2.0,10.897519,997,4:44,67,2017-06-30
13264,16888,Family Feud,1428,"Yeah, Dedication 6, of course\n Shout out Guru...","Yeah, yeah, yeah, yeah\n Six shit\n I could ca...",0.543581289923758,3.0,6.143931,1428,Dedication 6: Reloaded,94,2018-01-26
23374,30534,Family Feud,2675,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,None,NaN,NaN,2675,4:44,1,2017-06-30
23575,30735,Family Feud,2689,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,None,NaN,NaN,2689,4:44,1,2017-06-30
23649,30809,Family Feud,2695,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,None,NaN,NaN,2695,4:44,1,2017-06-30
23722,30882,Family Feud,2702,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,None,NaN,NaN,2702,4:44,1,2017-06-30
23760,30920,Family Feud,2707,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,None,NaN,NaN,2707,4:44,1,2017-06-30
23850,31010,Family Feud,2714,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,(Ha-ya) My nigga got on\n (Ha-ya) My nigga got...,None,NaN,NaN,2714,4:44,67,2017-06-30


In [64]:
df=df_songs_merged[df_songs_merged['artist_id']== 67]
duplicate_prob = process.extract('Family Feud R', df['name_x'], scorer=fuzz.ratio)
search = [dup[0] for dup in duplicate_prob if dup[1] > 80]

In [65]:
search

['Family Feud', 'Family Feud']

In [20]:
all_duplicates=[]

In [30]:
duplicates = []
for song in songs:
    query = session.query(Song).filter(Song.id == song)[0]
    artist = query.album.artist.name
    name = query.name
    duplicates.append((name, artist, song))
all_duplicates.append(duplicates)


In [31]:
all_duplicates

[[('F.U.T.W', 'Jay-Z', 11904), ('F.U.T.W', '21 Savage', 30548)],
 [('Family Feud', 'Jay-Z', 11890), ('Family Feud', 'Lil Wayne', 16888)],
 [('Family Feud', 'Jay-Z', 11890), ('Family Feud', 'Jay-Z', 11890)]]

In [63]:
dups = [x.id for x in session.query(Song).filter(Song.name == 'Family Feud').all()]

In [70]:
artist_song_ids = [(y.id, y.name) for x in session.query(Artist).filter(Artist.id == 67).all()[0].albums for y in x.songs]

In [69]:
[x for x in artist_song_ids if x in dups]

[11890, 31010]

In [71]:
artist_song_ids

[(11885, 'Kill Jay Z'),
 (11886, 'The Story of O.J.'),
 (11887, 'Smile'),
 (11888, 'Caught Their Eyes'),
 (11889, '4:44'),
 (11890, 'Family Feud'),
 (11891, 'Bam'),
 (11892, 'Moonlight'),
 (11893, 'Marcy Me'),
 (11894, 'Legacy'),
 (11895, 'Adnis'),
 (11896, "Blue's Freestyle/We Family"),
 (11897, 'MaNyfaCedGod'),
 (11899, 'Holy Grail'),
 (11900, 'Picasso Baby'),
 (11901, 'Tom Ford'),
 (11902, 'FuckWithMeYouKnowIGotIt'),
 (11903, 'Oceans'),
 (11904, 'F.U.T.W'),
 (11905, 'Somewhereinamerica'),
 (11906, 'Crown'),
 (11907, 'Heaven'),
 (11908, 'Versus (Interlude)'),
 (11909, 'Part II (On The Run)'),
 (11910, 'Beach Is Better (Interlude)'),
 (11911, 'BBC'),
 (11912, 'Jay Z Blue'),
 (11913, 'La Familia'),
 (11914, 'Nickels and Dimes'),
 (11915, 'Open Letter'),
 (11919, 'Public Service Announcement'),
 (11920, 'Run This Town'),
 (11921, "'03 Bonnie & Clyde"),
 (11922, 'Encore'),
 (11923, 'I Just Wanna Love U (Give it 2 Me)'),
 (11924, 'Izzo (H.O.V.A.)'),
 (11925, 'D.O.A. (Death of Auto-Tune)')

In [73]:
process.extract('Snoopy Track', [y[1] for y in artist_song_ids], scorer=fuzz.ratio)

[('Snoopy Track', 100),
 ("People Talkin'", 48),
 ('Pop 4 Roc', 48),
 ('Reasonable Doubt Tracklist', 47),
 ('Holy Grail', 45)]